In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_curve
from scipy import optimize
import math as mt

Using TensorFlow backend.


In [2]:
ev_1 = pd.read_hdf('../../data0/users/jisaacson/TrackML/events_200.h5').iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,18,19]]
evv = ev_1.loc[(ev_1['volume_id']==8)]
a = [str(i) for i in np.arange(1131, 1171)]
ev = evv[evv['event_id'].isin(a)]
ev

,event_id,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tpx,tpy,tpz,nch0,nch1
13884,1131,13885,-30.164200,-10.485100,-426.817993,8,2,1,549441559737671682,-0.065624,-0.080928,-0.034764,3.0,2.0
13885,1131,13886,-30.367599,-9.903200,-424.893005,8,2,1,0,-368861.000000,175418.000000,-912781.000000,4.0,9.0
13886,1131,13887,-25.203300,-19.721500,-488.829987,8,2,2,549458670887378946,0.248575,-0.093330,-0.251629,8.0,7.0
13887,1131,13888,-22.718500,-24.501200,-475.597992,8,2,3,0,169090.000000,239139.000000,-956149.000000,2.0,10.0
13888,1131,13889,-21.256300,-25.205799,-473.839996,8,2,3,0,75202.601562,650716.000000,-755588.000000,2.0,4.0
13889,1131,13890,2.671180,-32.896198,-474.114990,8,2,5,0,36379.601562,-749525.000000,-660975.000000,2.0,4.0
13890,1131,13891,5.124220,-32.038502,-467.826996,8,2,5,225189464673095681,-0.096577,-0.253776,0.096730,4.0,2.0
13891,1131,13892,29.511999,-12.320100,-424.079010,8,2,7,612518136641490945,-0.080272,-0.265160,0.260860,21.0,17.0
13892,1131,13893,32.567699,3.610520,-431.597992,8,2,9,549447263454236673,-0.160048,-0.168005,0.081625,2.0,2.0
13893,1131,13894,24.860001,23.469299,-445.183990,8,2,11,729586438185684993,0.087471,0.176636,-0.005392,1.0,1.0


In [3]:
def eta_calculation(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r)
    eta = -np.log(np.tan(theta/2.0))
    return eta
etas = []
df = pd.DataFrame()
for i in range(len(ev)):
    dat = ev.iloc[i, :]
    eta = eta_calculation(dat['x'], dat['y'], dat['z'])
    if np.absolute(eta) <= 1.2:
        etas.append(eta)
        df = df.append(dat)
df.insert(2, 'eta', etas)
df.to_pickle('final_data7_cut.pkl')
df = pd.read_pickle('final_data7_cut.pkl')


In [4]:
particle_ids = df['particle_id'].unique()
df

,event_id,hit_id,eta,layer_id,module_id,nch0,nch1,particle_id,tpx,tpy,tpz,volume_id,x,y,z
15959,1131,15960.0,-0.646519,2.0,97.0,3.0,1.0,1.621382e+17,-0.642880,-0.027483,-0.087105,8.0,-33.511501,-0.911154,-23.215599
15961,1131,15962.0,-0.567554,2.0,97.0,1.0,2.0,8.827085e+17,-0.337509,-0.138780,-0.032378,8.0,-29.322300,-12.892900,-19.171499
15964,1131,15965.0,-0.836063,2.0,97.0,1.0,6.0,4.953973e+17,-1.333720,-0.288761,-2.463160,8.0,-31.517099,-6.615520,-30.172701
15965,1131,15966.0,0.019401,2.0,97.0,2.0,4.0,7.656151e+17,-0.502291,-0.146998,-0.651941,8.0,-30.463200,-9.629890,0.619876
15966,1131,15967.0,-0.227065,2.0,97.0,2.0,5.0,6.395116e+17,-0.534638,-0.137424,-0.934022,8.0,-31.208599,-7.497800,-7.350820
15967,1131,15968.0,-0.705759,2.0,97.0,2.0,9.0,3.467825e+17,-0.245112,-0.077753,-0.821376,8.0,-30.065100,-10.768400,-24.456900
15968,1131,15969.0,-0.917714,2.0,97.0,2.0,2.0,5.404526e+16,-0.531998,0.031036,0.220088,8.0,-34.297600,1.337190,-36.110699
15970,1131,15971.0,-1.087182,2.0,97.0,1.0,4.0,3.737989e+17,-0.288720,-0.101429,-0.307016,8.0,-29.800900,-11.524100,-41.996101
15971,1131,15972.0,-0.601924,2.0,97.0,2.0,3.0,5.629517e+17,-0.321659,-0.008164,-0.273018,8.0,-33.174702,-1.874690,-21.230301
15972,1131,15973.0,-0.030813,2.0,97.0,3.0,3.0,7.070805e+17,-0.109172,-0.131843,0.117793,8.0,-30.330700,-10.008600,-0.984300


In [5]:
dff = pd.DataFrame()
for i in particle_ids:
    par_info = df.loc[df['particle_id'] == i]
    event_indicator = pd.DataFrame(par_info['event_id'].value_counts().sort_index())
    
    event_ids = np.array([j for j in event_indicator.loc[event_indicator['event_id'] >= 3].index])
    for j in event_ids:
        particle_info = par_info.loc[par_info['event_id'] == j]
        layer_indicator = particle_info['layer_id'].value_counts().sort_index()
        if i != 0 and len(layer_indicator) >= 3 and (sum(np.array(layer_indicator.index)) == 12 or sum(np.array(layer_indicator.index)) == 20):
            dff = dff.append(particle_info)
##looking for event of a particle with at least 3 hits 
##for such event, check it has at least one hit per layer on layer_id 2, layer_id 4, and layer_id 6; 2+4+6 == 12 and 2+4+6+8 == 20 
dff.to_pickle('ideal_final_data7_cut.pkl')

In [6]:
ideal_particle_ids = dff['particle_id'].unique()
dff

,event_id,hit_id,eta,layer_id,module_id,nch0,nch1,particle_id,tpx,tpy,tpz,volume_id,x,y,z
15959,1131,15960.0,-0.646519,2.0,97.0,3.0,1.0,1.621382e+17,-0.642880,-0.027483,-0.087105,8.0,-33.511501,-0.911154,-23.215599
17304,1131,17305.0,-0.667373,2.0,112.0,1.0,2.0,1.621382e+17,-0.642958,-0.026402,-0.087248,8.0,-32.032600,-0.840086,-23.008200
23406,1131,23407.0,-0.383055,4.0,193.0,1.0,1.0,1.621382e+17,-0.640863,-0.049976,-0.088387,8.0,-72.835602,-3.236530,-28.615601
29511,1131,29512.0,-0.293873,6.0,313.0,1.0,1.0,1.621382e+17,-0.637565,-0.076682,-0.089418,8.0,-115.930000,-7.551220,-34.634399
34971,1131,34972.0,-0.243260,8.0,470.0,2.0,1.0,1.621382e+17,-0.632118,-0.110654,-0.088645,8.0,-172.311996,-15.824800,-42.509399
15961,1131,15962.0,-0.567554,2.0,97.0,1.0,2.0,8.827085e+17,-0.337509,-0.138780,-0.032378,8.0,-29.322300,-12.892900,-19.171499
16115,1131,16116.0,-0.547738,2.0,98.0,2.0,1.0,8.827085e+17,-0.336887,-0.137861,-0.032539,8.0,-30.762699,-13.497400,-19.334400
23421,1131,23422.0,-0.308945,4.0,194.0,1.0,1.0,8.827085e+17,-0.344370,-0.116828,-0.031118,8.0,-66.405502,-26.862801,-22.484400
23445,1131,23446.0,-0.302989,4.0,195.0,3.0,2.0,8.827085e+17,-0.344906,-0.115278,-0.030499,8.0,-68.472198,-27.551001,-22.706499
29540,1131,29541.0,-0.226089,6.0,315.0,2.0,1.0,8.827085e+17,-0.351439,-0.090545,-0.031789,8.0,-108.714996,-39.382999,-26.365601


In [7]:
ideal_ch1 = []
ideal_x = []
ideal_y = []
ideal_z = []

In [8]:
def ideal_data(par_id):
    par_info = dff.loc[dff['particle_id']==par_id]
    ideal_ch1 = []
    ideal_x = []
    ideal_y = []
    ideal_z = []
    for s in np.unique(par_info['event_id']):
        particle_info = par_info.loc[par_info['event_id'] == s]   
        par_lay2, par_lay4, par_lay6 = particle_info.loc[particle_info['layer_id'] == 2], particle_info.loc[particle_info['layer_id'] == 4], particle_info.loc[particle_info['layer_id'] == 6]
        for i in range(len(par_lay2)):
            for j in range(len(par_lay4)):
                for k in range(len(par_lay6)):
                    data_lay2, data_lay4, data_lay6 = par_lay2.iloc[i,:], par_lay4.iloc[j,:], par_lay6.iloc[k, :] 
                    x_lay2, x_lay4, x_lay6 = data_lay2['x'], data_lay4['x'], data_lay6['x']
                    y_lay2, y_lay4, y_lay6 = data_lay2['y'], data_lay4['y'], data_lay6['y']
                    z_lay2, z_lay4, z_lay6 = data_lay2['z'], data_lay4['z'], data_lay6['z']
                    ch_lay2, ch_lay4, ch_lay6 = data_lay2['nch1'], data_lay4['nch1'], data_lay6['nch1']

                    ch1_1 = [ch_lay2, ch_lay4, ch_lay6]
                    x_1 = [x_lay2, x_lay4, x_lay6]
                    y_1 = [y_lay2, y_lay4, y_lay6]
                    z_1 = [z_lay2, z_lay4, z_lay6]
                    ideal_ch1.append(ch1_1)
                    ideal_x.append(x_1)
                    ideal_y.append(y_1)
                    ideal_z.append(z_1)
    return ideal_ch1, ideal_x, ideal_y, ideal_z



In [9]:
a, b, c, d = ideal_data(1.0358499045277696e+17)

In [10]:
a

[[1.0, 1.0, 1.0], [1.0, 2.0, 2.0], [2.0, 2.0, 1.0], [1.0, 2.0, 1.0]]

In [11]:
b

[[-32.528499603271484, -70.99539947509766, -113.96900177001953],
 [-31.67020034790039, -71.03260040283203, -114.8010025024414],
 [21.240999221801758, 46.975399017333984, 73.15699768066406],
 [21.995500564575195, 46.975399017333984, 73.15699768066406]]

In [12]:
c

[[-3.7227799892425537, -10.816300392150879, -22.086700439453125],
 [-4.677519798278809, -8.564390182495117, -10.61870002746582],
 [23.414199829101562, 54.575401306152344, 90.09120178222656],
 [24.299999237060547, 54.575401306152344, 90.09120178222656]]

In [13]:
d

[[-25.971900939941406, -17.534400939941406, -8.140629768371582],
 [6.115620136260986, 17.00429916381836, 29.118099212646484],
 [33.84339904785156, 22.240100860595703, 9.434379577636719],
 [33.5093994140625, 22.240100860595703, 9.434379577636719]]

In [14]:
dff.loc[dff['particle_id']==1.0358499045277696e+17]

,event_id,hit_id,eta,layer_id,module_id,nch0,nch1,particle_id,tpx,tpy,tpz,volume_id,x,y,z
16039,1131,16040.0,-0.727394,2.0,97.0,2.0,1.0,1.035850e+17,-0.341868,-0.050015,0.074649,8.0,-32.528500,-3.722780,-25.971901
23411,1131,23412.0,-0.241799,4.0,193.0,1.0,1.0,1.035850e+17,-0.336085,-0.073940,0.074309,8.0,-70.995399,-10.816300,-17.534401
29528,1131,29529.0,-0.070066,6.0,314.0,1.0,1.0,1.035850e+17,-0.329059,-0.100388,0.072091,8.0,-113.969002,-22.086700,-8.140630
35450,1131,35451.0,0.019978,8.0,549.0,2.0,1.0,1.035850e+17,-0.317544,-0.130892,0.071883,8.0,-166.074005,-40.561401,3.415630
35453,1131,35454.0,0.022032,8.0,550.0,1.0,1.0,1.035850e+17,-0.316722,-0.132452,0.072455,8.0,-167.878998,-41.300201,3.809380
25317,1139,25318.0,0.189888,2.0,113.0,3.0,1.0,1.035850e+17,-0.464198,-0.058188,0.125113,8.0,-31.670200,-4.677520,6.115620
34439,1139,34440.0,0.235484,4.0,225.0,2.0,2.0,1.035850e+17,-0.464509,-0.033912,0.128858,8.0,-71.032600,-8.564390,17.004299
42379,1139,42380.0,0.249951,6.0,365.0,2.0,2.0,1.035850e+17,-0.465382,-0.008699,0.127982,8.0,-114.801003,-10.618700,29.118099
49471,1139,49472.0,0.258433,8.0,547.0,2.0,2.0,1.035850e+17,-0.463665,0.026211,0.128848,8.0,-171.141006,-9.493920,44.791199
23401,1141,23402.0,0.930384,2.0,122.0,1.0,2.0,1.035850e+17,0.297001,0.342575,-0.127641,8.0,21.240999,23.414200,33.843399


In [15]:
ideal_particle_ids[0]

1.6213817651992986e+17

In [16]:
ideal_data(1.6213817651992986e+17)

([[1.0, 1.0, 1.0], [2.0, 1.0, 1.0]],
 [[-33.51150131225586, -72.83560180664062, -115.93000030517578],
  [-32.03260040283203, -72.83560180664062, -115.93000030517578]],
 [[-0.9111539721488953, -3.236530065536499, -7.551219940185547],
  [-0.8400859832763672, -3.236530065536499, -7.551219940185547]],
 [[-23.215599060058594, -28.6156005859375, -34.6343994140625],
  [-23.00819969177246, -28.6156005859375, -34.6343994140625]])

In [17]:
ideal_x

[]

In [18]:
for par_id in ideal_particle_ids:    
    par_info = dff.loc[dff['particle_id']==par_id] 
    ch1s, xs, ys, zs = ideal_data(par_id)
    for ch1 in ch1s:
        ideal_ch1.append(ch1)
    for x in xs:
        ideal_x.append(x)
    for y in ys:
        ideal_y.append(y)
    for z in zs:
        ideal_z.append(z)

In [19]:
len(ideal_ch1)

112143

In [20]:
len(dff['particle_id'].unique())

33678

In [21]:
ideal_ch1

[[1.0, 1.0, 1.0],
 [2.0, 1.0, 1.0],
 [2.0, 1.0, 1.0],
 [2.0, 2.0, 1.0],
 [1.0, 1.0, 1.0],
 [1.0, 2.0, 1.0],
 [4.0, 3.0, 4.0],
 [4.0, 3.0, 4.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [5.0, 4.0, 4.0],
 [5.0, 4.0, 5.0],
 [5.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 5.0],
 [4.0, 4.0, 4.0],
 [2.0, 2.0, 2.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [1.0, 2.0, 1.0],
 [5.0, 6.0, 5.0],
 [3.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0],
 [1.0, 2.0, 1.0],
 [5.0, 6.0, 5.0],
 [5.0, 6.0, 5.0],
 [2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0],
 [4.0, 4.0, 4.0],
 [3.0, 4.0, 4.0],
 [6.0, 5.0, 6.0],
 [6.0, 5.0, 6.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 2.0],
 [3.0, 4.0, 3.0],
 [3.0, 4.0, 3.0],
 [2.0, 2.0, 1.0],
 [3.0, 3.0, 4.0],
 [3.0, 2.0, 2.0],
 [3.0, 3.0, 2.0],
 [3.0, 2.0, 2.0],
 [3.0, 3.0, 2.0],
 [2.0, 2.0, 2.0],
 [2.0, 3.0, 2.0],
 [1.0, 1.0, 1.0],
 [3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0],
 [2.0, 2.0, 1.0],
 [3.0, 3.0, 3.0],
 [1.0, 2.0, 2.0],
 [1.0, 2.0, 2.0],
 [4.0, 4.0

In [22]:
np.save('f_ideal_ch1_number_7_cut.npy', np.array(ideal_ch1))
np.save('f_ideal_x_number_7_cut.npy', np.array(ideal_x))
np.save('f_ideal_y_number_7_cut.npy', np.array(ideal_y))
np.save('f_ideal_z_number_7_cut.npy', np.array(ideal_z))


In [23]:
ideal_x

[[-33.51150131225586, -72.83560180664062, -115.93000030517578],
 [-32.03260040283203, -72.83560180664062, -115.93000030517578],
 [-29.32229995727539, -66.40550231933594, -108.71499633789062],
 [-29.32229995727539, -68.47219848632812, -108.71499633789062],
 [-30.762699127197266, -66.40550231933594, -108.71499633789062],
 [-30.762699127197266, -68.47219848632812, -108.71499633789062],
 [6.0894999504089355, 11.173600196838379, 13.67959976196289],
 [6.0894999504089355, 11.173600196838379, 13.717599868774414],
 [23.399900436401367, 52.87409973144531, 86.7123031616211],
 [25.148500442504883, 52.87409973144531, 86.7123031616211],
 [29.655099868774414, 66.32689666748047, 107.60299682617188],
 [29.655099868774414, 66.32689666748047, 108.0739974975586],
 [29.655099868774414, 66.32689666748047, 109.98899841308594],
 [31.5137996673584, 66.32689666748047, 107.60299682617188],
 [31.5137996673584, 66.32689666748047, 108.0739974975586],
 [31.5137996673584, 66.32689666748047, 109.98899841308594],
 [-25

In [38]:
ideal_ch1 = np.load('f_ideal_ch1_number_7_cut.npy')
ideal_x = np.load('f_ideal_x_number_7_cut.npy')
ideal_y = np.load('f_ideal_y_number_7_cut.npy')
ideal_z = np.load('f_ideal_z_number_7_cut.npy')
len(ideal_ch1)

112143

In [25]:
43554 /40

1088.85

In [26]:
def difference_parameters(hit1, hit2):
    x1, y1, z1 = hit1[0], hit1[1], hit1[2]
    x2, y2, z2 = hit2[0], hit2[1], hit2[2]
    r1, r2 = np.sqrt(x1**2 + y1**2), np.sqrt(x2**2 + y2**2)
    phi1, phi2 = np.arctan2(y1, x1), np.arctan2(y2, x2)
    dphi = phi2 - phi1
    if dphi > np.pi:
        dphi -= 2*np.pi
    
    elif dphi < -np.pi:
        dphi += 2*np.pi
    dz = z2 - z1
    dr = r2 - r1
    
    phi_slope = dphi/dr
    z0 = z1 - r1 * dz/dr
    
    R1, R2 = np.sqrt(r1**2 + z1**2), np.sqrt(r2**2 + z2**2)
    theta1, theta2 = np.arccos(z1/R1), np.arccos(z2/R2)    
    eta1, eta2 = -np.log(np.tan(theta1/2.0)), -np.log(np.tan(theta2/2.0))
    deta = eta2 - eta1
    return [dphi, dz, dr, phi_slope, z0, deta]

def stronger_difference_parameters(hit1, hit2):
    x1, y1, z1 = hit1[0], hit1[1], hit1[2]
    x2, y2, z2 = hit2[0], hit2[1], hit2[2]
    r1, r2 = np.sqrt(x1**2 + y1**2), np.sqrt(x2**2 + y2**2)
    phi1, phi2 = np.arctan2(y1, x1), np.arctan2(y2, x2)
    dphi = phi2 - phi1
    if dphi > np.pi:
        dphi -= 2*np.pi
        
    elif dphi < -np.pi:
        dphi += 2*np.pi
    dz = z2 - z1
    dr = r2 - r1
    
    phi_slope = dphi/dr
    z0 = z1 - r1 * dz/dr
    
    R1, R2 = np.sqrt(r1**2 + z1**2), np.sqrt(r2**2 + z2**2)
    theta1, theta2 = np.arccos(z1/R1), np.arccos(z2/R2)    
    eta1, eta2 = -np.log(np.tan(theta1/2.0)), -np.log(np.tan(theta2/2.0))
    deta = eta2 - eta1
    return [dphi, dz, dr, phi_slope, z0, deta, z1, r1, phi1, eta1]

In [27]:
# contains the x, y, z coordinates of a hit on layer_2, layer_4, layer_6
ideals_layer2 = np.array([list(ideal_x[:,0]), list(ideal_y[:,0]), list(ideal_z[:,0])]).transpose()
ideals_layer4 = np.array([list(ideal_x[:,1]), list(ideal_y[:,1]), list(ideal_z[:,1])]).transpose()
ideals_layer6 = np.array([list(ideal_x[:,2]), list(ideal_y[:,2]), list(ideal_z[:,2])]).transpose()

In [28]:
ideal_x

array([[ -33.51150131,  -72.83560181, -115.93000031],
       [ -32.0326004 ,  -72.83560181, -115.93000031],
       [ -29.32229996,  -66.40550232, -108.71499634],
       ...,
       [ -31.13800049,  -69.63670349, -113.69799805],
       [ -31.13800049,  -71.73870087, -113.69799805],
       [ -30.96389961,  -71.03630066, -111.15200043]])

In [29]:
ideal_y

array([[ -0.91115397,  -3.23653007,  -7.55121994],
       [ -0.84008598,  -3.23653007,  -7.55121994],
       [-12.89290047, -26.8628006 , -39.38299942],
       ...,
       [  6.22912979,  14.56040001,  24.6548996 ],
       [  6.22912979,  15.01760006,  24.6548996 ],
       [  9.31532955, -16.88500023, -34.23400116]])

In [30]:
ideal_z

array([[-23.21559906, -28.61560059, -34.63439941],
       [-23.00819969, -28.61560059, -34.63439941],
       [-19.17149925, -22.4843998 , -26.36560059],
       ...,
       [  2.5718801 ,   7.56546021,  13.25940037],
       [  2.5718801 ,   7.81839991,  13.25940037],
       [ 23.54789925,  -3.4511199 , -28.81570053]])

In [31]:
ideals_layer2

array([[-33.51150131,  -0.91115397, -23.21559906],
       [-32.0326004 ,  -0.84008598, -23.00819969],
       [-29.32229996, -12.89290047, -19.17149925],
       ...,
       [-31.13800049,   6.22912979,   2.5718801 ],
       [-31.13800049,   6.22912979,   2.5718801 ],
       [-30.96389961,   9.31532955,  23.54789925]])

In [32]:
ideals_layer4

array([[-72.83560181,  -3.23653007, -28.61560059],
       [-72.83560181,  -3.23653007, -28.61560059],
       [-66.40550232, -26.8628006 , -22.4843998 ],
       ...,
       [-69.63670349,  14.56040001,   7.56546021],
       [-71.73870087,  15.01760006,   7.81839991],
       [-71.03630066, -16.88500023,  -3.4511199 ]])

In [34]:
ideal_difference_layer24 = []
for i, j in zip(ideals_layer2, ideals_layer4):
    ideal_24 = difference_parameters(i, j)
    ideal_difference_layer24.append(ideal_24)
ideal_difference_layer24 = np.array(ideal_difference_layer24)

In [40]:
ideal_difference_layer46 = []
for i, j in zip(ideals_layer4, ideals_layer6):
    ideal_46 = stronger_difference_parameters(i, j)
    ideal_difference_layer46.append(ideal_46)
ideal_difference_layer46 = np.array(ideal_difference_layer46)
np.save('f_ideal_diff_lay24_7_cut.npy', np.array(ideal_difference_layer24))
np.save('f_ideal_diff_lay46_7_cut.npy', np.array(ideal_difference_layer46))

In [41]:
ideal_difference_layer24 = np.load('f_ideal_diff_lay24_7_cut.npy')
ideal_difference_layer46 = np.load('f_ideal_diff_lay46_7_cut.npy')
ideal_difference_layer46

array([[ 2.06372589e-02, -6.01879883e+00,  4.32681925e+01, ...,
         7.29074757e+01, -3.09718577e+00, -3.83055321e-01],
       [ 2.06372589e-02, -6.01879883e+00,  4.32681925e+01, ...,
         7.29074757e+01, -3.09718577e+00, -3.83055321e-01],
       [-3.68485551e-02, -3.88120079e+00,  4.39954926e+01, ...,
         7.16330985e+01, -2.75719003e+00, -3.08944693e-01],
       ...,
       [-7.41813396e-03,  5.69394016e+00,  4.51978002e+01, ...,
         7.11426435e+01,  2.93547133e+00,  1.06142712e-01],
       [-7.18186454e-03,  5.44100046e+00,  4.30467228e+01, ...,
         7.32937208e+01,  2.93523506e+00,  1.06470891e-01],
       [ 6.54085546e-02, -2.53645806e+01,  4.32890158e+01, ...,
         7.30154726e+01, -2.90822792e+00, -4.72480158e-02]])

In [42]:
len(ideal_difference_layer24)

112143

In [ ]:
#indicator_test = np.array(list(trues_test) + list(falses_test))
actual_true = np.array([x for x, y in zip(decision_score,indicator_test) if y==1])
actual_false = np.array([x for x, y in zip(decision_score,indicator_test) if y==0])

In [ ]:
def roc_values(decision_scores):
    cuts = np.unique(decision_scores)
    tprs, fprs = [], []
    special_cuts, efficiencies, purities = [], [], []
    for cut in cuts:
        true_positive = np.array([i for i in actual_true if i >= cut])
        false_positive = np.array([i for i in actual_false if i >= cut])
        tpr, fpr = len(true_positive)/len(actual_true), len(false_positive)/len(actual_false)
        
        if abs(tpr-0.98) <= 0.02:
            efficiencies.append(tpr)
            purity = 1/(1+fpr/tpr*len(falses_test)/len(trues_test))
            purities.append(purity)
            special_cuts.append(cut)
            
        tprs.append(tpr)
        fprs.append(fpr)
    return np.array(tprs), np.array(fprs), np.array(special_cuts), np.array(efficiencies), np.array(purities)

In [ ]:
def purity_evaluation(tpr, fpr):
    purity = []
    for i in range(len(tpr)):
        if tpr[i] == 0:
            purity.append(0)
        else:
            purity.append(1/(1+fpr[i]/tpr[i]*len(falses_test)/len(trues_test)))
    return np.array(purity)